In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer, \
                         TextIteratorStreamer, TrainingArguments, \
                         DataCollatorForLanguageModeling, Trainer, \
                         pipeline
from datasets import load_dataset
from peft import PeftModel, LoraConfig, TaskType, get_peft_model

### Load base model

In [ ]:
model_base_name = "Llama-3.2-3B-Instruct"
model_name = f"meta-llama/{model_base_name}"
base_model = AutoModelForCausalLM.from_pretrained(model_name) #"Qwen/Qwen2-0.5B-Instruct")
tokenizer = AutoTokenizer.from_pretrained(model_name) #"Qwen/Qwen2-0.5B-Instruct")  # Changed to load from fine-tuned path
tokenizer.pad_token=tokenizer.eos_token

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

### Create PEFT config

In [ ]:
peft_config = LoraConfig(task_type=TaskType.CAUSAL_LM, inference_mode=False, r=8, lora_alpha=32, lora_dropout=0.1)
peft_model = get_peft_model(base_model, peft_config)

peft_model.num_parameters()
peft_model.print_trainable_parameters()

### Training Setup

In [ ]:
target_model_path = f"./finetuned/{model_name}-lora"

training_args = TrainingArguments(
    output_dir=f"{target_model_path}/",
    learning_rate=5e-4,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=2,
    num_train_epochs=10,
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    fp16=True,
    gradient_accumulation_steps=16,
    logging_steps=500,
    weight_decay=0.01,
    warmup_steps=500,
)

### Load Dataset

In [ ]:
dataset = load_dataset("Hieu-Pham/kaggle_food_recipes", cache_dir=".")
dataset = [ str(data["Instructions"]) for data in dataset["train"] ]
#print(dataset[0])
context_length=2048

dataset = tokenizer(dataset, add_special_tokens=True, truncation=True, max_length=context_length)["input_ids"]
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [ ]:
trainer = Trainer(
    model=peft_model,
    args=training_args,
    train_dataset=dataset,
    eval_dataset=dataset,
    tokenizer=tokenizer,
    data_collator=data_collator,
    #compute_metrics=compute_metrics,
)

trainer.train()

In [ ]:
pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer)
max_new_tokens = 1024
while True:
    prompt = input(">>> Prompt: ")
    if prompt == "quit":
        break
    print(pipe(prompt, max_new_tokens=max_new_tokens)[0]["generated_text"], "\n")